<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Variational-Autoencoder-Using-Keras" data-toc-modified-id="Variational-Autoencoder-Using-Keras-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Variational Autoencoder Using Keras</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Loading-Data" data-toc-modified-id="Loading-Data-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Loading Data</a></span></li></ul></li></ul></li></ul></div>

# Variational Autoencoder Using Keras

The code is based on the book [Generative Deep Learning by David Foster](https://www.oreilly.com/library/view/generative-deep-learning/9781492041931/).

In [1]:
from Models.VAE_Keras import VariationalAutoencoder_Keras

Using TensorFlow backend.
/home/nahum/anaconda3/envs/generative/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nahum/anaconda3/envs/generative/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nahum/anaconda3/envs/generative/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nahum

In [3]:
import os 

# run params
SECTION = 'vae'
RUN_ID = '0001'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.makedirs(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build'

In [5]:
vae = VariationalAutoencoder_Keras(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64]#, 64]
    , encoder_conv_kernel_size = [3,3,3]#,3]
    , encoder_conv_strides = [1,2,2]#,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

W0116 20:01:44.087090 139686552659776 deprecation_wrapper.py:119] From /home/nahum/anaconda3/envs/generative/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0116 20:01:44.119557 139686552659776 deprecation_wrapper.py:119] From /home/nahum/anaconda3/envs/generative/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0116 20:01:44.139746 139686552659776 deprecation_wrapper.py:119] From /home/nahum/anaconda3/envs/generative/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0116 20:01:44.304983 139686552659776 deprecation_wrapper.py:119] From /home/nahum/anaconda3/envs/generative/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_norma

### Loading Data

In [6]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))

In [7]:
vae.encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
leaky_re_l

In [8]:
vae.decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0         
__________

In [9]:
#Compile Parameters

LEARNING_RATE = 0.0005
R_LOSS_FACTOR = 1000

In [10]:
vae.compile(LEARNING_RATE, R_LOSS_FACTOR)

W0116 20:01:49.624658 139686552659776 deprecation_wrapper.py:119] From /home/nahum/anaconda3/envs/generative/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [11]:
# Training Parameters

BATCH_SIZE = 32
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [ ]:
vae.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

W0116 20:01:52.109563 139686552659776 deprecation_wrapper.py:119] From /home/nahum/anaconda3/envs/generative/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/200
60000/60000 [==============================] - 189s 3ms/step - loss: 59.4454 - vae_r_loss: 56.4007 - vae_kl_loss: 3.0447

Epoch 00001: saving model to run/vae/0001_digits/weights/weights-001-59.45.h5

Epoch 00001: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 2/200
60000/60000 [==============================] - 189s 3ms/step - loss: 52.9575 - vae_r_loss: 49.1738 - vae_kl_loss: 3.7838

Epoch 00002: saving model to run/vae/0001_digits/weights/weights-002-52.96.h5

Epoch 00002: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 3/200
60000/60000 [==============================] - 185s 3ms/step - loss: 51.5100 - vae_r_loss: 47.4509 - vae_kl_loss: 4.0591

Epoch 00003: saving model to run/vae/0001_digits/weights/weights-003-51.51.h5

Epoch 00003: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 4/200
60000/60000 [==============================] - 185s 3ms/step - loss: 50.8006 - vae_r_loss: 46.6181 - vae_kl_loss: 4.1825

Epoch 00004: saving mod